In [49]:
import requests
import json
from bs4 import BeautifulSoup
#import bs4
import re
import pandas as pd 
import plotly.express as px
import plotly as py
import plotly.graph_objs as go

In [39]:
page_url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population' 
response=requests.get(page_url)
print(response.status_code)

200


In [40]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})

df=pd.read_html(str(indiatable))
# convert list to dataframe
df=pd.DataFrame(df[0])
#clean data
df['Country / Dependency'] = df['Country / Dependency'].str.replace("\[b\]", "")
data = df.drop(["Date", "Source (official or from the United Nations)","Notes","Percentage of the world"], axis=1)  

print(data.head(20))

   Rank Country / Dependency    Region  Population
0     –                World       NaN  7936719000
1     1                China      Asia  1412600000
2     2                India      Asia  1374271375
3     3        United States  Americas   332564727
4     4            Indonesia      Asia   272248500
5     5             Pakistan      Asia   225200000
6     6               Brazil  Americas   214385501
7     7              Nigeria    Africa   211401000
8     8           Bangladesh      Asia   172383258
9     9               Russia    Europe   145478097
10   10               Mexico  Americas   126014024
11   11                Japan      Asia   125340000
12   12             Ethiopia    Africa   117876000
13   13          Philippines      Asia   111623674
14   14                Egypt    Africa   103049323
15   15              Vietnam      Asia    98510000
16   16             DR Congo    Africa    92378000
17   17                 Iran      Asia    85276270
18   18               Turkey   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning:

The default value of regex will change from True to False in a future version.



In [41]:
#check nan
df.isnull().sum()

Rank                                              0
Country / Dependency                              0
Region                                            1
Population                                        0
Percentage of the world                         241
Date                                              0
Source (official or from the United Nations)      0
Notes                                           208
dtype: int64

In [42]:
fig = go.Figure(data=go.Choropleth(
    locations = df['Country / Dependency'],locationmode = "country names",z = df['Population'],
    text = df['Country / Dependency'],colorscale = 'Reds',autocolorscale=False,reversescale=True,
    marker_line_color='darkgray',marker_line_width=0.5,colorbar_tickprefix = '',colorbar_title = 'Population',
))
fig.update_layout(
    title_text='Population of Countries ',geo=dict(
        showframe=False,showcoastlines=True,projection_type='equirectangular' ),
    annotations = [dict( x=0.55, y=0.1, xref='paper', yref='paper',text=' ',showarrow = False )] )
fig.show()

In [46]:
#plotlyexpress
fig=px.choropleth(df, locations = df['Country / Dependency'],locationmode = "country names", color="Population", hover_name="Country / Dependency",
                  color_continuous_scale=px.colors.diverging.BrBG,labels={'Country / Dependency':'Country'},title="Population of Countries")
fig.show()

In [55]:
pageurl="https://en.wikipedia.org/wiki/List_of_states_and_union_territories_of_India_by_population"
response=requests.get(pageurl)
print(response.status_code)

200


In [63]:
sp = BeautifulSoup(response.text, 'html.parser')
indiatable=sp.find('table',{'class':"wikitable"})

df=pd.read_html(str(indiatable))
# convert list to dataframe
df=pd.DataFrame(df[0])
df.rename(columns = {'Density[a]':'Density'}, inplace = True)
df.head()

,Rank,State or union territory,Population,National Share (%),Decadal growth(2001–2012),Rural population,Percent rural,Urban population,Percent urban,Area[14],Density,Sex ratio
0,1 (S1),Uttar Pradesh,199812341,NaN,20.2%,155317278,NaN,44495063,NaN,"240,928 km2 (93,023 sq mi)","828/km2 (2,140/sq mi)",912
1,2 (S2),Maharashtra,112374333,NaN,20.0%,61556074,NaN,50818259,NaN,"307,713 km2 (118,809 sq mi)",365/km2 (950/sq mi),929
2,3 (S3),Bihar,104099452,NaN,25.4%,92341436,NaN,11758016,NaN,"94,163 km2 (36,357 sq mi)","1,102/km2 (2,850/sq mi)",918
3,4 (S4),West Bengal,91276115,NaN,13.8%,62183113,NaN,29093002,NaN,"88,752 km2 (34,267 sq mi)","1,029/km2 (2,670/sq mi)",953
4,5 (S5),Madhya Pradesh,72626809,NaN,16.3%,52557404,NaN,20069405,NaN,"308,245 km2 (119,014 sq mi)",236/km2 (610/sq mi),931


In [35]:
df.isnull().sum()

Rank                        0
City                        0
Population(2011)[3]         0
Population(2001)            0
State or union territory    0
dtype: int64

In [64]:
#india_states = json.load(open("states_india.geojson", "r"))
fig = px.choropleth(
    df,
    locationmode='geojson-id', featureidkey='properties.ST_NM',
    locations=df['State or union territory'],
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    color="Density",
    hover_name="State or union territory",
    hover_data=["Population","Rural population","Urban population"],
    title="India Population Density",
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()